* 정배열 : 단기 이동평균선이 가장 위에, 장기 이동평균선이 가장 아래에 위치하도록 이평선들이 순서대로 배치된 상태
* 역배열 : 정배열과 반대로 배치된 상태

## 정배열이 시작되는 지점에서 매수, 끝나는 지점에서 매도할때의 기대수익 계산

In [2]:
import pandas as pd

stock_info = pd.read_excel("data/211104_시가총액.xlsx")  # 데이터는 깃허브에서 다운로드받았음

# 코스피, 코스닥, 시총 상위 100개의 우량주로 데이터 구분
KOSPI_list = stock_info.loc[stock_info["시장구분"] == "KOSPI", "종목명"].tolist()
KOSDAQ_list = stock_info.loc[stock_info["시장구분"] == "KOSDAQ", "종목명"].tolist()
TOP100_list = stock_info.sort_values(by = "시가총액", ascending = False)['종목명'].iloc[:100].tolist()

In [3]:
import os
from tqdm import tqdm

sp_data_dict = dict()
for file_name in tqdm(os.listdir("data/주가데이터")):
    stock_name = file_name.replace('.csv', '')
    sp_data = pd.read_csv("data/주가데이터/" + file_name, usecols = ["Date", "Close"], parse_dates = ['Date'])
    sp_data_dict[stock_name] = sp_data

100%|██████████| 2170/2170 [00:12<00:00, 168.24it/s]


In [4]:
# 정배열, 역배열 생성

for stock_name in sp_data_dict.keys():
    sp_data = sp_data_dict[stock_name]
    sp_data["5일선"] = sp_data["Close"].rolling(5).mean()
    sp_data["20일선"] = sp_data["Close"].rolling(20).mean()
    sp_data["60일선"] = sp_data["Close"].rolling(60).mean()
    sp_data["120일선"] = sp_data["Close"].rolling(120).mean()
    sp_data.dropna(inplace = True)

    sp_data["정배열"] = (sp_data["5일선"] >= sp_data["20일선"])\
                          & (sp_data["20일선"] >= sp_data["60일선"])\
                          & (sp_data["60일선"] >= sp_data["120일선"])

    sp_data["역배열"] = (sp_data["5일선"] <= sp_data["20일선"])\
                          & (sp_data["20일선"] <= sp_data["60일선"])\
                          & (sp_data["60일선"] <= sp_data["120일선"])

In [8]:
# 정배열이 시작될 때 매수하고 정배열이 끝나는 시점에 매도하는 함수

import numpy as np

def calc_ror_in_forward(sp_data):
    ror_list = []

    # ex) [2일 종가, 3일 종가, 4일 종가] > [1일 종가, 2일 종가, 3일 종가] 이렇게 비교하며 True인 날 (정배열이 아니었다가 정배열이 된 날)이 매수시점
    buy_point_list = sp_data["정배열"].values[1:] > sp_data["정배열"].values[:-1]
    buy_point_list = np.insert(buy_point_list, 0, False)  # 1일차는 비교대상이 없으므로 정배열이 새로 발생한건지 알 수 없음. 따라서 매수x
    buy_point_list = sp_data.index[buy_point_list]

    sell_point_list = sp_data["정배열"].values[:-1] > sp_data["정배열"].values[1:]
    sell_point_list = np.insert(sell_point_list, 0, False)
    sell_point_list = sp_data.index[sell_point_list]

    for buy_point in buy_point_list:
        if (sum(buy_point<sell_point_list) > 0) and (buy_point+1 <= sp_data.index[-1]):
            buy_price = sp_data.loc[buy_point + 1, "Close"]
            sell_point = sell_point_list[sell_point_list > buy_point][0] + 1
            if sell_point <= sp_data.index[-1]:
                sell_price = sp_data.loc[sell_point, "Close"]

                ror = (sell_price - buy_price) / buy_price * 100
                ror_list.append(ror)
        else:
            break

    return ror_list

In [9]:
total_ror_list = []

for stock_name in sp_data_dict.keys():
    sp_data = sp_data_dict[stock_name]
    ror_list = calc_ror_in_forward(sp_data)
    total_ror_list += ror_list
    break

print(pd.Series(total_ror_list).describe().round(2))

# 평균이 2.42로 양수이지만 중위수가 -1.57이고 max가 비정상적으로 높기 때문에 평균적으로 음수인 결과가 많은것을 알 수 있음
# 따라서 좋은 전략은 아님

count    28.00
mean      2.42
std      16.92
min     -14.96
25%      -6.23
50%      -1.57
75%       1.16
max      58.17
dtype: float64


In [10]:
# 정배열이 시작될 때 매수하고 역배열이 시작될 때 매도하는 함수

def calc_ror_in_for_rev(sp_data):
    ror_list = []
    buy_point_list = sp_data["정배열"].values[1:] > sp_data["정배열"].values[:-1]
    buy_point_list = np.insert(buy_point_list, 0, False)
    buy_point_list = sp_data.index[buy_point_list]

    sell_point_list = sp_data["역배열"].values[1:] > sp_data["역배열"].values[:-1]
    sell_point_list = np.insert(sell_point_list, 0, False)
    sell_point_list = sp_data.index[sell_point_list]

    for buy_point in buy_point_list:
        if (sum(buy_point<sell_point_list) > 0) and (buy_point+1<=sp_data.index[-1]):
            buy_price = sp_data.loc[buy_point + 1, "Close"]
            sell_point = sell_point_list[sell_point_list > buy_point][0] + 1
            if sell_point <= sp_data.index[-1]:
                sell_price = sp_data.loc[sell_point, "Close"]

                ror = (sell_price - buy_price) / buy_price * 100
                ror_list.append(ror)
        else:
            break

    return ror_list

In [11]:
total_ror_list = []

for stock_name in sp_data_dict.keys():
    sp_data = sp_data_dict[stock_name]
    ror_list = calc_ror_in_for_rev(sp_data)
    total_ror_list += ror_list

print(pd.Series(total_ror_list).describe().round(2))

# 평균은 더 높아졌으나 중위수도 더 낮아짐. 정배열 시작 - 정배열 종료 전략에 비해서 수익과 손실 모두 클 수 있음을 의미

count    52122.00
mean         4.28
std         57.55
min        -95.39
25%        -17.62
50%         -8.35
75%          6.34
max       2537.21
dtype: float64


## 정배열과 역배열을 사용한 전략은 알려진 바와 다르게 수익률이 좋지 못했음
## 코드상으로는 전체 종목을 대상으로 했지만 코스닥, 코스피, 우량주의 경우 모두 동일하게 좋지 못한 결과를 보임